# 通用数据处理

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_load_process_common_data.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_load_process_common_data.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/dataset_load_process/load_process_common_data.ipynb)

## 概述

数据是深度学习的基础，良好的数据输入可以对整个深度神经网络训练起到非常积极的作用。在训练前对已加载的数据集进行数据处理，可以解决诸如数据量过大、样本分布不均等问题，从而获得对训练结果更有利的数据输入。

MindSpore的各个数据集类都为用户提供了多种数据处理操作，用户可以通过构建数据处理pipeline来定义需要使用的数据处理操作，在训练过程中，数据即可像水一样源源不断地经过数据处理pipeline流向训练系统。

MindSpore目前支持的部分常用数据处理操作如下表所示，更多数据处理操作参见[API文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.dataset.html)。

| 数据处理操作  | 操作说明 |
| :----  | :----           |
| shuffle | 对数据集进行混洗，随机打乱数据顺序。 |
| map | 提供自定义函数，作用于数据集的指定列数据。 |
| batch | 对数据集进行分批，可以减少训练轮次，加速训练过程。 |
| repeat | 对数据集进行重复，达到扩充数据量的目的。 |
| zip | 将两个数据集进行列拼接，合并为一个数据集。 |
| concat | 将两个数据集进行行拼接，合并为一个数据集。 |

## 数据处理操作

### shuffle

shuffle会随机打乱数据顺序，对数据集进行混洗。设定的`buffer_size`越大，混洗程度越大，同时所消耗的时间、计算资源也更大。

![shuffle](https://gitee.com/mindspore/docs/raw/master/docs/mindspore/programming_guide/source_zh_cn/images/shuffle.png)

下面的样例先构建了一个随机数据集，然后对其进行混洗操作，最后展示了数据混洗前后的结果。

In [37]:
import numpy as np
import mindspore.dataset as ds

ds.config.set_seed(0)

def generator_func():
    for i in range(5):
        yield (np.array([i, i+1, i+2]),)

dataset = ds.GeneratorDataset(generator_func, ["data"])
for data in dataset.create_dict_iterator():
    print(data)

print("------ after processing ------")

dataset = dataset.shuffle(buffer_size=2)
for data in dataset.create_dict_iterator():
    print(data)

{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}
------ after processing ------
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}


### map

map将指定的函数作用于数据集的指定列数据，实现数据映射操作。用户可以自定义映射函数，也可以直接使用`c_transforms`或`py_transforms`中的函数针对图像、文本数据进行数据增强。更多数据增强的使用方式，请参见：[图像数据加载与增强](https://mindspore.cn/tutorials/zh-CN/master/intermediate/dataset_load_process/image_data_augmentation.html)和[文本数据加载与增强](https://mindspore.cn/tutorials/zh-CN/master/intermediate/dataset_load_process/text_data_augmentation.html)章节。

![map](https://gitee.com/mindspore/docs/raw/master/docs/mindspore/programming_guide/source_zh_cn/images/map.png)

下面的样例先构建了一个随机数据集，然后定义了数据翻倍的映射函数并将其作用于数据集，最后对比展示了映射前后的数据结果。

In [38]:
import numpy as np
import mindspore.dataset as ds

def generator_func():
    for i in range(5):
        yield (np.array([i, i+1, i+2]),)

def pyfunc(x):
    return x*2

dataset = ds.GeneratorDataset(generator_func, ["data"])
for data in dataset.create_dict_iterator():
    print(data)

print("------ after processing ------")

dataset = dataset.map(operations=pyfunc, input_columns=["data"])

for data in dataset.create_dict_iterator():
    print(data)

{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}
------ after processing ------
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 2, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 4, 6])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 6, 8])}
{'data': Tensor(shape=[3], dtype=Int64, value= [ 6,  8, 10])}
{'data': Tensor(shape=[3], dtype=Int64, value= [ 8, 10, 12])}


### batch

batch将数据集分批，分别输入到训练系统中进行训练，可以减少训练轮次，达到加速训练过程的目的。

![batch](https://gitee.com/mindspore/docs/raw/master/docs/mindspore/programming_guide/source_zh_cn/images/batch.png)

下面的样例先构建了一个数据集，然后分别展示了丢弃多余数据与否的数据集分批结果，其中批大小为2。

In [39]:
import numpy as np
import mindspore.dataset as ds

def generator_func():
    for i in range(5):
        yield (np.array([i, i+1, i+2]),)

dataset = ds.GeneratorDataset(generator_func, ["data"])
for data in dataset.create_dict_iterator():
    print(data)

dataset = dataset.batch(batch_size=2, drop_remainder=False)
print("------not drop remainder ------")
for data in dataset.create_dict_iterator():
    print(data)

dataset = ds.GeneratorDataset(generator_func, ["data"])
dataset = dataset.batch(batch_size=2, drop_remainder=True)
print("------ drop remainder ------")
for data in dataset.create_dict_iterator():
    print(data)

{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}
------not drop remainder ------
{'data': Tensor(shape=[2, 3], dtype=Int64, value=
[[0, 1, 2],
 [1, 2, 3]])}
{'data': Tensor(shape=[2, 3], dtype=Int64, value=
[[2, 3, 4],
 [3, 4, 5]])}
{'data': Tensor(shape=[1, 3], dtype=Int64, value=
[[4, 5, 6]])}
------ drop remainder ------
{'data': Tensor(shape=[2, 3], dtype=Int64, value=
[[0, 1, 2],
 [1, 2, 3]])}
{'data': Tensor(shape=[2, 3], dtype=Int64, value=
[[2, 3, 4],
 [3, 4, 5]])}


### repeat

repeat对数据集进行重复，达到扩充数据量的目的。`repeat`和`batch`操作的先后顺序会影响训练batch的数量，建议将`repeat`置于`batch`之后。

![repeat](https://gitee.com/mindspore/docs/raw/master/docs/mindspore/programming_guide/source_zh_cn/images/repeat.png)

下面的样例先构建了一个随机数据集，然后将其重复2次，最后展示了重复后的数据结果。

In [40]:
import numpy as np
import mindspore.dataset as ds

def generator_func():
    for i in range(5):
        yield (np.array([i, i+1, i+2]),)

dataset = ds.GeneratorDataset(generator_func, ["data"])
for data in dataset.create_dict_iterator():
    print(data)

dataset = dataset.repeat(count=2)
print("------ after processing ------")
for data in dataset.create_dict_iterator():
    print(data)

{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}
------ after processing ------
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}


### zip

zip实现两个数据集的列拼接，将其合并为一个数据集。使用时需要注意以下两点：

1. 如果两个数据集的列名相同，则不会合并，请注意列的命名。
2. 如果两个数据集的行数不同，合并后的行数将和较小行数保持一致。

![zip](https://gitee.com/mindspore/docs/raw/master/docs/mindspore/programming_guide/source_zh_cn/images/zip.png)

下面的样例先构建了两个不同样本数的随机数据集，然后将其进行列拼接，最后展示了拼接后的数据结果。

In [41]:
import numpy as np
import mindspore.dataset as ds

def generator_func():
    for i in range(7):
        yield (np.array([i, i+1, i+2]),)

def generator_func2():
    for _ in range(4):
        yield (np.array([1, 2]),)

dataset1 = ds.GeneratorDataset(generator_func, ["data1"])
print("data1:")
for data in dataset1.create_dict_iterator():
    print(data)
dataset2 = ds.GeneratorDataset(generator_func2, ["data2"])
print("data2:")
for data in dataset2.create_dict_iterator():
    print(data)

dataset3 = ds.zip((dataset1, dataset2))
print("data3:")
for data in dataset3.create_dict_iterator():
    print(data)

data1:
{'data1': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [3, 4, 5])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [4, 5, 6])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [5, 6, 7])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [6, 7, 8])}
data2:
{'data2': Tensor(shape=[2], dtype=Int64, value= [1, 2])}
{'data2': Tensor(shape=[2], dtype=Int64, value= [1, 2])}
{'data2': Tensor(shape=[2], dtype=Int64, value= [1, 2])}
{'data2': Tensor(shape=[2], dtype=Int64, value= [1, 2])}
data3:
{'data1': Tensor(shape=[3], dtype=Int64, value= [0, 1, 2]), 'data2': Tensor(shape=[2], dtype=Int64, value= [1, 2])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3]), 'data2': Tensor(shape=[2], dtype=Int64, value= [1, 2])}
{'data1': Tensor(shape=[3], dtype=Int64, value= [2, 3, 4]), 'data2': Tensor(shape=[2], dtype=Int64,

### concat

concat实现两个数据集的行拼接，并将其合并为一个数据集。使用时需要注意：输入数据集中的列名、列数据类型和列数据的排列应相同。

![concat](https://gitee.com/mindspore/docs/raw/master/docs/mindspore/programming_guide/source_zh_cn/images/concat.png)

下面的样例先构建了两个随机数据集，然后将其做行拼接，最后展示了拼接后的数据结果。值得一提的是，使用`+`运算符也能达到同样的效果。

In [42]:
import numpy as np
import mindspore.dataset as ds

def generator_func():
    for _ in range(2):
        yield (np.array([0, 0, 0]),)

def generator_func2():
    for _ in range(2):
        yield (np.array([1, 2, 3]),)

dataset1 = ds.GeneratorDataset(generator_func, ["data"])
print("data1:")
for data in dataset1.create_dict_iterator():
    print(data)
dataset2 = ds.GeneratorDataset(generator_func2, ["data"])
print("data2:")
for data in dataset2.create_dict_iterator():
    print(data)

dataset3 = dataset1.concat(dataset2)
print("data3:")
for data in dataset3.create_dict_iterator():
    print(data)

data1:
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 0, 0])}
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 0, 0])}
data2:
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
data3:
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 0, 0])}
{'data': Tensor(shape=[3], dtype=Int64, value= [0, 0, 0])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
{'data': Tensor(shape=[3], dtype=Int64, value= [1, 2, 3])}
